make sure the SNR is actually being changed  
and test out the new encoder set up  
the idea is that we have a bunch of neurons that are gonna fire at the same firing rate
then I would then calculate the mean and standard deviation. 

In [1]:
import numpy as np
np.set_printoptions(precision=2, suppress=True)


percent_of_variance_in_a_list = np.arange(0, 1.3, 0.1)[:, np.newaxis]

print(f'set up the variances in a list:')

print(percent_of_variance_in_a_list)

n_neurons = len(percent_of_variance_in_a_list)

print(f'we therefore know the number of neurons to be {n_neurons}')


set up the variances in a list:
[[0. ]
 [0.1]
 [0.2]
 [0.3]
 [0.4]
 [0.5]
 [0.6]
 [0.7]
 [0.8]
 [0.9]
 [1. ]
 [1.1]
 [1.2]]
we therefore know the number of neurons to be 13


In [2]:
from features.simulation_features import get_enc_setup

print(f'we need to validate that the generated firing rates, should be different')
print(f'we meed to set up the encoder')
      
ENCODER_TYPE = 'cosine_tuned_encoder'

#neuron set up : 'std (20 neurons)' or 'toy (4 neurons)' 
#N_NEURONS, N_STATES, sim_C = get_enc_setup(sim_mode = 'rot_90',
#                                          n_neurons = n_neurons)

N_NEURONS = n_neurons
N_STATES = 7
X_VEL_IND = 3
MEAN_FIRING_RATE = 100

sim_C = np.zeros((N_NEURONS, N_STATES))

sim_C[:,X_VEL_IND] = MEAN_FIRING_RATE

we need to validate that the generated firing rates, should be different
we meed to set up the encoder


/home/sijia-aw/BMi3D_my/lab_bmi3d/riglib/blackrock/cerelink.py:13: UserWarning: Unable to import cerebus library. Check if is installed if using the Blackrock NeuroPort system
  warnings.warn("Unable to import cerebus library. Check if is installed if using the Blackrock NeuroPort system")


In [3]:
from features.simulation_features import SimCosineTunedEnc
from riglib.bmi.state_space_models import StateSpaceEndptVel2D
from riglib.bmi.sim_neurons import GenericCosEnc

ssm = StateSpaceEndptVel2D()


generic_cosine_enc = GenericCosEnc(sim_C, ssm)

state_sample = np.array([0,0,0,1,0,0,0])[:,np.newaxis]


print(f'at the heart of this algorithm is that')
print(f'we are generating poisson spikes at rates determined by sim_C and ssm ')
print(f' multiplied by the interval of {generic_cosine_enc.DT} s')

generic_cosine_enc(state_sample)

at the heart of this algorithm is that
we are generating poisson spikes at rates determined by sim_C and ssm 
 multiplied by the interval of 0.1 s


array([[11],
       [10],
       [ 9],
       [ 7],
       [12],
       [10],
       [16],
       [15],
       [ 9],
       [14],
       [ 6],
       [ 8],
       [10]])

# Validate new encoder with added noises

In [4]:
from riglib.bmi.sim_neurons import GenericCosEncWithNoise

cos_enc_poisson_noise = GenericCosEncWithNoise(sim_C, ssm, noise_mode = 'relative_gaussian',
                                                 noise_profile= percent_of_variance_in_a_list)
sim_C

<bound method GenericCosEncWithNoise._gen_relative_gaussian_noise of <riglib.bmi.sim_neurons.GenericCosEncWithNoise object at 0x7f8579f57e20>>


array([[  0.,   0.,   0., 100.,   0.,   0.,   0.],
       [  0.,   0.,   0., 100.,   0.,   0.,   0.],
       [  0.,   0.,   0., 100.,   0.,   0.,   0.],
       [  0.,   0.,   0., 100.,   0.,   0.,   0.],
       [  0.,   0.,   0., 100.,   0.,   0.,   0.],
       [  0.,   0.,   0., 100.,   0.,   0.,   0.],
       [  0.,   0.,   0., 100.,   0.,   0.,   0.],
       [  0.,   0.,   0., 100.,   0.,   0.,   0.],
       [  0.,   0.,   0., 100.,   0.,   0.,   0.],
       [  0.,   0.,   0., 100.,   0.,   0.,   0.],
       [  0.,   0.,   0., 100.,   0.,   0.,   0.],
       [  0.,   0.,   0., 100.,   0.,   0.,   0.],
       [  0.,   0.,   0., 100.,   0.,   0.,   0.]])

In [18]:
print(f'uisng the same state sample')

state_sample = np.array([0,0,0,10,0,0,0])[:,np.newaxis]

x = cos_enc_poisson_noise(state_sample)
x

uisng the same state sample
[[111]
 [ 86]
 [100]
 [ 78]
 [110]
 [ 98]
 [ 94]
 [ 85]
 [102]
 [ 94]
 [ 87]
 [117]
 [ 90]]


array([[111.  ],
       [ 96.  ],
       [ 98.91],
       [ 70.37],
       [127.55],
       [ 69.54],
       [155.11],
       [ 81.17],
       [ 86.9 ],
       [  0.  ],
       [186.79],
       [184.04],
       [198.45]])

# calculate the residue